In [1]:
# !pip install evaluate datasets

In [2]:
# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

In [3]:
import numpy as np
import evaluate

from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_from_disk
from evaluate import evaluator

In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:256"

In [5]:
#!unzip lid.zip
ds = load_from_disk('lid').with_format("torch")

In [6]:
langs = ds['test'].features['label'].names

In [7]:
id2label = {idx: langs[idx] for idx in range(len(langs))}
label2id = {v: k for k, v in id2label.items()}

In [8]:
tokenizer = AutoTokenizer.from_pretrained("google/canine-c")

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


In [9]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding='max_length')

tokenized_ds = ds.map(preprocess_function, batched=True)

In [12]:
train_ds = tokenized_ds['train'].shuffle(seed=42).select(range(1000))
test_ds = tokenized_ds['test'].shuffle(seed=42).select(range(200))

In [13]:
model = AutoModelForSequenceClassification.from_pretrained("google/canine-c", num_labels=len(langs), id2label=id2label, label2id=label2id)

Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-c and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()